In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dropout
%matplotlib inline  
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras import layers, models

In [2]:
def prepareData():
    allData = pd.read_csv("data/songs_with_genre.csv", index_col=0,
                          header=[0, 1, 2])
    generes = allData["genre_id"]
    genres_index = pd.read_csv("data/genres_index.csv", index_col=0)
    allData.drop(columns=["genre_id"], inplace=True)
    data_x = allData
    data_x.columns = range(data_x.shape[1])
    #normalizing
    mu = np.mean(data_x, axis=0)
    sigma = (np.std(data_x, axis=0))
    data_x = (data_x - mu) / sigma
    #Creating the right lables
    generes.columns = range(generes.shape[1])
    targets = OneHotEncoder().fit_transform(genres_index).toarray()
    generes[0] = generes[0] - 1
    data_y = [targets[i - 1] for i in generes[0]]
    generes[0] = data_y
    
    test = generes.copy()
    test.rename(columns={0: "oneHotVector"}, inplace=True)
    test.shape
    split = pd.DataFrame(test["oneHotVector"].to_list(), columns=range(161))
    split.index = generes.index
    generes = split
    return data_x, generes

In [3]:
data_x, labels = prepareData()
x_train, x_rest, y_train, y_rest = train_test_split(data_x, labels, train_size=0.8)
x_test, x_valid, y_test, y_valid = train_test_split(x_rest, y_rest, train_size=0.5)
print(x_train.shape), print(len(y_train))
print(x_valid.shape), print(len(y_valid))
print(x_test.shape), print(len(y_test))

(39678, 518)
39678
(4960, 518)
4960
(4960, 518)
4960


(None, None)

In [14]:
n_features, n_outputs = x_train.shape[1], y_train.shape[1]
CNN = models.Sequential()
CNN.add(layers.Conv1D(filters=32, kernel_size=5, activation='relu',
                      input_shape=(n_features, 1), padding='same'))
CNN.add(layers.MaxPooling1D(pool_size=4))
CNN.add(layers.Conv1D(filters=64, kernel_size=7, activation='relu'))
# CNN.add(layers.Dropout(0.4))
CNN.add(layers.MaxPooling1D(pool_size=4))
CNN.add(layers.Conv1D(filters=128, kernel_size=7, activation='relu'))
CNN.add(layers.Dropout(0.7))
CNN.add(layers.MaxPooling1D(pool_size=2))
CNN.add(layers.Flatten())
CNN.add(layers.Dense(100, activation='relu'))
CNN.add(layers.Dense(n_outputs, activation='softmax'))

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_18. Consider increasing the input size. Received input shape [None, 518, 1, 64] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
CNN.summary()

In [ ]:
CNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# training = CNN.fit(x_train, y_train,epochs=10, batch_size=102)
training = CNN.fit(x_train, y_train, epochs=30, validation_data=(x_valid,y_valid), batch_size=389)


In [ ]:
_, accuracy = CNN.evaluate(x_valid, y_valid, verbose=0)
print(accuracy)